# Shortest Path Example

9.3-1. You need to take a trip by car to another town that you have never visited before. Therefore, you are studying a map to determine the shortest route to your destination. Depending on which route you choose, there are five other towns (call them A, B, C, D, E) that you might pass through on the way. The map
shows the mileage along each road that directly connects two towns without any intervening towns. These numbers are sum- marized in the following table, where a dash indicates that there is no road directly connecting these two towns without going through any other towns.

This problem is from Chapter 3, Section 1 (3.1 Prototype Example) of the Introduction to Operations Research,
7th edition by Hillier and Lieberman.

<div>
<img src="img_9.3-1_table.png" width="400"/>
</div>

### Graph Notation

<!-- ![title](img_9.3-1_network.png) -->
<div>
<img src="img_9.3-1_network.png" width="700"/>
</div>

Let V represent the set of vertices in the graph:
$$ V =\{O,A,B,C,D,E,T\} $$

Let E represent the set of edges in the graph:

$$ 
E = \{OA, OB, OC, AB, AD, BD, BE, CB, CE, DE, DT, ET\}
$$

Let $u$ represent the first vertice and $v$ represent the second vertice in edge $UV$ in the path $UV-VW$ (U->V->W)

Let $v$ represent the first vertice and $w$ represent the second vertice in edge $VW$ in the path $UV-VW$ (U->V->W)

Let $x_{uv}$ $\epsilon$ $(0,1)$
<br>$x_{uv}$ = 1 if edge ${uv}$ is used, 0 otherwise.

Let $d_{uv}$ represent the distance between edge $u$ and $v$.

### Mathematical Formulation

$Minimize: Z $



$$Z = \sum_{u,v \: \epsilon \:E} x_{uv}$$

$ 
s.t: 
$

Constraint 1: Ensures path starts at origin and that each subsequent edge in the path is a continuation from the previous edge. In other words, if the trip enters into a given city (excluding final destination), it must leave the same city on the next leg of the path. 
<br>
<br>
$$
\sum_{u,v \: \epsilon \:E} x_{uv} = 
\sum_{v,w \: \epsilon \:E} x_{uv} \qquad 
\forall  \enspace v \: \epsilon \:V
$$ 



Constraint 2: Ensures the path reaches the final destination. 
<br>
<br>
$$
\sum_{u,t \: \epsilon \:E} x_{ut} = 1 
$$

# DOCplex.mp Python Model

### Install packages (only if needed)

Import using conda in the current Jupyter kernel

In [1]:
# import sys
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} matplotlib
# !conda install --yes --prefix {sys.prefix} docplex

Or alternitively, import a using pip

In [2]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install docplex

### Import packages

In [3]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model
import docplex.mp.solution as Solution

### Intializing the data

In [4]:
# Initialize the problem data
df = pd.read_csv('data_arc.csv')
df

,origin,destination,OD_pair,distance
0,O,A,OA,40
1,O,C,OC,60
2,O,B,OB,50
3,A,B,AB,10
4,A,D,AD,70
5,B,D,BD,20
6,B,E,BE,55
7,C,B,CB,40
8,C,E,CE,50
9,D,E,DE,10


In [5]:
edges = list((t.origin, t.destination) for t in df.titertuples())
edges

AttributeError: 'DataFrame' object has no attribute 'titertuples'

In [ ]:
last_arc_df = df[df['destination'] == 'T']
last_arc = list((t.origin, t.destination) for t in last_arc_df.itertuples())
last_arc

In [ ]:
final_destination = 'T'
cities = set(df['destination'])
cities.remove(final_destination)
cities

In [ ]:

# cities = ['A', 'B','C', 'D', 'E']


In [ ]:
distance = dict([((t.origin, t.destination),t.distance ) for t in df.itertuples()])
distance

### Create the model

In [ ]:
m = Model('Shortest_Path')

recall:
Let $x_{uv}$ $\epsilon$ $(0,1)$
<br>$x_{uv}$ = 1 if edge ${uv}$ is used, 0 otherwise.

In [ ]:
# Create decision variables
x = m.binary_var_dict(edges, name = 'x')


#### <font color=green>  Objective Function

recall: Minimize Z $$Z = \sum_{u,v } x_{uv}$$

In [ ]:
m.minimize(m.sum(distance[uv]*x[uv] for uv in edges))
print(m.export_to_string())

#### <font color=green>  Constraints

Recall: Constraint 1:
$$
\sum_{u,v \: \epsilon \:E} x_{uv} = 
\sum_{v,w \: \epsilon \:E} x_{uv} \qquad 
\forall  \enspace v \: \epsilon \: Cities
$$ 



Recall: Constraint 2:
$$
\sum_{u,t \: \epsilon \:E} x_{ut} = 1 
$$

In [ ]:
# constraint 1
for c in cities:
    m.add_constraint((m.sum(x[(u,v)] for u,v in edges if v==c))== 
                     (m.sum(x[(v,w)] for v,w in edges if v==c)), ctname='city_'+ c)

# constraint 2
m.add_constraint(m.sum(x[(i,j)] for i,j in last_arc if j=='T')== 1, ctname='last_arc')

In [ ]:
print(m.export_to_string())

### Solve

In [ ]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

soln = m.solve(log_output=True)

In [ ]:
print(m.get_solve_status())


In [ ]:
soln.display()

In [ ]:
lst = []
for i in x:
    if soln.get_var_value(x[i]) >0:
        lst.append(i[1])
lst



In [ ]:
lst = []
for u,v in edges:
    if soln.get_var_value(x[u,v])> 0:
        arc = u+'->'+v
        print(arc)
        solution = (u, v, arc,distance[u,v])
        lst.append(solution)
lst


In [ ]:
df = pd.DataFrame.from_records(lst, columns=['starting_city', 'next_city','arc_name','distance'])
df